<a href="https://colab.research.google.com/github/sathsaraRasantha/Question_Answering_for_Sinhala_Language_using_Deep_NN_Architectures/blob/main/Preprocessing_Pipeline_for_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import pandas as pd
import pickle
import re, os, string, typing, gc, json
import spacy
from collections import Counter
nlp = spacy.load('en')

In [2]:
from google.colab import files
uploaded = files.upload()

Saving english_sinhala_combined.csv to english_sinhala_combined.csv


In [3]:
data=pd.read_csv("english_sinhala_combined.csv")
data=data.drop(['Unnamed: 0',	'context',	'question',	'text'	],axis=1)

In [4]:
answer_start=[]
answer_end=[]

for i in range(0,20000):
  text=str(data['context_translated'][i])
  answer=str(data['text_translated'][i])
  start=text.find(answer)
  end=start+len(answer)
  answer_start.append(start)
  answer_end.append(end)

In [5]:
data['answer_start']=answer_start
data['answer_end']=answer_end
data

,context_translated,question_translated,text_translated,answer_start,answer_end
0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්,0,17
1,"2002 දී ප්‍රදර්ශනය ආරම්භ වූ දා සිට, අයිඩල් ජයග...",ඇමරිකානු අයිඩල් තරඟකරු ක්‍රිස් ඩොට්‍රි කුමන ප්...,උතුරු කැරොලිනාව,-1,14
2,3 වන ශතවර්ෂයේ මැද භාගයේ සිට ජර්මානු ගෝත්‍රිකයන...,7 වන සියවසේ ඉදිකරන ලද දේ,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි,-1,44
3,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි ...,90 දශකයේ දී කැනේ විසින් සාදන ලද බීට්ස් කවුද?,දේශීය කලාකරුවන්,-1,14
4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්,58,68
...,...,...,...,...,...
19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ...",27,103
19996,පැරණි ඉංග්‍රීසි භාෂාවෙන් ලතින් භාෂාවෙන් යම් නි...,ලතින් හෝඩිය හඳුන්වාදීමට පෙර පැරණි ඉංග්‍රීසි ලි...,රූනික් පද්ධතිය,-1,13
19997,රූනික් පද්ධතිය... ඇරිස්ටෝෆනේස්ගේ නාට්‍ය ග්‍රීක...,ප්ලේටෝ සහ ඇරිස්ටෝටල් ලියුවේ කුමන ආකාරයේ සාහිත්...,දාර්ශනික පා,-1,10
19998,ආරම්භක හා අවසන් දිනයන් දළ වශයෙන් දකුණු අර්ධගෝල...,දකුණු අර්ධගෝලයේ කුමන කාල පරිච්ඡේදයේදී චිලී ප්‍...,ගිම්හානය,312,320


In [6]:
df = data[data.answer_start != -1]
df

,context_translated,question_translated,text_translated,answer_start,answer_end
0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්,0,17
4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්,58,68
8,ජේන් ඔස්ටන්... ප්‍රධාන තදාසන්න ප්‍රදේශ අතර නිර...,රිච්මන්ඩ් හි නිරිතදිග තදාසන්න ප්‍රදේශය කුමක්ද?,මිඩ්ලෝතියන්,56,67
9,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනිය මිත්‍ර පාක්ෂිකය...,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනියේ මුලින් ප්‍රාන්...,හත,339,341
21,නැපෝලියන් පශ්චාත් විප්ලවවාදී ප්‍රංශයේ අවනීතිය ...,දහස් ගණනකට යුද්ධය හා මරණය පිළිබඳ අපේක්ෂාවෙන් න...,ඔහුගේ විවේචකයෝ,164,178
...,...,...,...,...,...
19990,"1900 වන විට මිනිසුන් 7,531 ක් නගරයේ වාසය කළහ. ...",2006 දී ටක්සන්ගේ ජනගහනය කොපමණද?,"535,000 කි",528,538
19991,තෙවන පරම්පරාව ෆයර්වෙයාර් හෝ යූඑස්බී සම්බන්ධතාව...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,320,325
19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට 10 ...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් වන ස්වාභාවික ර...,සර්කැඩියානු රිද්මය,133,151
19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ...",27,103


In [7]:
df=df.reset_index()
df

,index,context_translated,question_translated,text_translated,answer_start,answer_end
0,0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්,0,17
1,4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්,58,68
2,8,ජේන් ඔස්ටන්... ප්‍රධාන තදාසන්න ප්‍රදේශ අතර නිර...,රිච්මන්ඩ් හි නිරිතදිග තදාසන්න ප්‍රදේශය කුමක්ද?,මිඩ්ලෝතියන්,56,67
3,9,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනිය මිත්‍ර පාක්ෂිකය...,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනියේ මුලින් ප්‍රාන්...,හත,339,341
4,21,නැපෝලියන් පශ්චාත් විප්ලවවාදී ප්‍රංශයේ අවනීතිය ...,දහස් ගණනකට යුද්ධය හා මරණය පිළිබඳ අපේක්ෂාවෙන් න...,ඔහුගේ විවේචකයෝ,164,178
...,...,...,...,...,...,...
8993,19990,"1900 වන විට මිනිසුන් 7,531 ක් නගරයේ වාසය කළහ. ...",2006 දී ටක්සන්ගේ ජනගහනය කොපමණද?,"535,000 කි",528,538
8994,19991,තෙවන පරම්පරාව ෆයර්වෙයාර් හෝ යූඑස්බී සම්බන්ධතාව...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,320,325
8995,19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට 10 ...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් වන ස්වාභාවික ර...,සර්කැඩියානු රිද්මය,133,151
8996,19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ...",27,103


In [8]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [9]:
df.to_csv('indexed_data.csv')
!cp indexed_data.csv "drive/My Drive/"

In [10]:
df

,index,context_translated,question_translated,text_translated,answer_start,answer_end
0,0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්,0,17
1,4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්,58,68
2,8,ජේන් ඔස්ටන්... ප්‍රධාන තදාසන්න ප්‍රදේශ අතර නිර...,රිච්මන්ඩ් හි නිරිතදිග තදාසන්න ප්‍රදේශය කුමක්ද?,මිඩ්ලෝතියන්,56,67
3,9,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනිය මිත්‍ර පාක්ෂිකය...,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනියේ මුලින් ප්‍රාන්...,හත,339,341
4,21,නැපෝලියන් පශ්චාත් විප්ලවවාදී ප්‍රංශයේ අවනීතිය ...,දහස් ගණනකට යුද්ධය හා මරණය පිළිබඳ අපේක්ෂාවෙන් න...,ඔහුගේ විවේචකයෝ,164,178
...,...,...,...,...,...,...
8993,19990,"1900 වන විට මිනිසුන් 7,531 ක් නගරයේ වාසය කළහ. ...",2006 දී ටක්සන්ගේ ජනගහනය කොපමණද?,"535,000 කි",528,538
8994,19991,තෙවන පරම්පරාව ෆයර්වෙයාර් හෝ යූඑස්බී සම්බන්ධතාව...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,320,325
8995,19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට 10 ...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් වන ස්වාභාවික ර...,සර්කැඩියානු රිද්මය,133,151
8996,19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ...",27,103


In [15]:
def parse_data(dataframe):
  QA_list=[]

  for i in range(0,len(dataframe['index'])):
    QA_dict={}
    QA_dict['id']=i
    QA_dict['context']=dataframe['context_translated'][i]
    QA_dict['question']=dataframe['question_translated'][i]
    QA_dict['label']=[dataframe['answer_start'][0],dataframe['answer_end'][i]]
    QA_list.append(QA_dict)
  return QA_list



In [31]:
def gather_text_for_vocab(df):
    
    text = []
    total = 0
    
    unique_contexts = list(df.context_translated.unique())
    unique_questions = list(df.question_translated.unique())
    total += df.context_translated.nunique() + df.question_translated.nunique()
    text.extend(unique_contexts + unique_questions)
    
    #assert len(text) == total
    
    return text

In [32]:
gather_text_for_vocab(df)

['උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහිටීම එහි උෂ්ණත්වය පාලනය කරයි, නමුත් බොහෝ හිම සහ වැසි ඇති කළ හැකි නොර් ඊස්ටර් කාලගුණ පද්ධති වලට නගරය බෙහෙවින් නැඹුරු වේ. නගරයේ සාමාන්\u200dය වර්ෂයකට අඟල් 43.8 (මි.මී. 1,110) ක වර්ෂාපතනයක් ඇති අතර එක් කන්නයකට අඟල් 43.8 (සෙ.මී. 111) හිම පතනයක් සිදු වේ. යමෙකු නගරයෙන් (විශේෂයෙන් නගරයට උතුරින් හා බටහිරින්) away තට යන විට හිම පතනය නාටකාකාර ලෙස ඉහළ යයි. එය සාගරයේ මධ්\u200dයස්ථ බලපෑමෙන් away ත්වේ. බොහෝ හිම පතනය සිදුවන්නේ දෙසැම්බර් සිට මාර්තු දක්වා වන අතර බොහෝ වසරවල අප්රේල් සහ නොවැම්බර් මාසවල මැනිය හැකි හිම නොමැති අතර මැයි සහ ඔක්තෝබර් මාසවල හිම දුර්ලභ වේ. හිම පතනයෙහි ඉහළ සිට වසර දක්වා විචල්\u200dයතාවයක් ද ඇත; නිදසුනක් ලෙස, 2011-12 ශීත saw තුව දුටුවේ හිම සමුච්චය 9.3 (සෙ.මී. 23.6) ක් පමණි, නමුත් පෙර ශීත, තුවේ දී අනුරූප අගය 81.0 (මීටර් 2.06) විය. []] []]',
 'ටුවාලුආන් භාෂාව සහ ඉංග්\u200dරීසි යනු ටුවාලු හි ජාතික භාෂා වේ. ටුවාලුවාන් යනු පොලිනීසියානු භාෂාවන්හි එලිසියානු කණ්ඩායමට අයත් වන අතර හවායි, මාඕරි, ටහීටියානු, සැමෝවාන් සහ ටොංගන් වැනි අනෙකුත් සියලුම ප